In [ ]:
import os
import time
import shutil
from datetime import datetime, timedelta
from IPython.display import clear_output

# ANSI color codes
RED = "\033[91m"
GREEN = "\033[92m"
YELLOW = "\033[93m"
DARK_GREEN = "\033[38;5;213m"
RESET = "\033[0m"

def copy_files(src_dir, temp_dir):
    """Copy all files from source dir to temp dir (fresh snapshot)."""
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)  # remove old snapcshot
    os.makedirs(temp_dir, exist_ok=True)

    for filename in os.listdir(src_dir):
        src_file = os.path.join(src_dir, filename)
        dst_file = os.path.join(temp_dir, filename)
        if os.path.isfile(src_file):
            shutil.copy2(src_file, dst_file)

def tail_last_line(file_path):
    """Extract progress and error info from log file."""
    try:
        with open(file_path, 'r', errors="ignore") as f:
            valid = f.read().split("\n")
        valid_lines = [x.split("Working on Match")[-1].strip(" ").strip(".") 
                       for x in valid if "Working on Match" in x]
        error_lines = [x.split("INFO: Type     :")[-1].strip(" ") 
                       for x in valid if "INFO: Type     :" in x] + \
                      [x.split("Type         =>")[-1].strip(" ") 
                       for x in valid if "Type         =>" in x]
        pre_done_lines = [x.split("i.e.")[-1].strip(" ").strip(".").split(" ")[0].strip('.')
                       for x in valid if "Existing file already has the same number of fixtures with event links" in x]
        
        if pre_done_lines:
            count = pre_done_lines[0]
            return f"{count}/{count}","Done Already"
        if len(valid_lines) == 0 and len(error_lines) == 0 and len(pre_done_lines):
            return None,None

        valid_last = valid_lines[-1] if valid_lines else "0/1"
        error_last = error_lines[-1] if error_lines else ""
        return valid_last, error_last
    except Exception:
        return "0/1", ""

def watch_directory(src_dir, interval=7):
    temp_dir = os.path.join(src_dir, "_temp_snapshot")
    last_progress = {}  # remember progress from last check

    try:
        # Copy logs into temp dir
        while True:
            copy_files(src_dir, temp_dir)
    
            rows = []
            stale_count=0
            for filename in os.listdir(temp_dir):
                filepath = os.path.join(temp_dir, filename)
                if os.path.isfile(filepath):
                    valid, err = tail_last_line(filepath)
                    if valid is None and err is None :
                        continue
                    try:
                        done, total = [int(x) for x in valid.split("/")]
                        percent = (done / total) * 100 if total > 0 else 0
                        progress_str = f"{percent:.2f}% ({valid})"
                    except Exception:
                        done, total, percent = 0, 1, 0
                        progress_str = "0.00% (0/1)"
                    
                    #remive stale logs
                    # mtime = datetime.fromtimestamp(os.path.getmtime(filepath))
                    # recent_threshold = datetime.now() - timedelta(hours=30)
                    # is_recent = mtime >= recent_threshold
                    # if not is_recent:
                    #     stale_count += 1
                    #     continue  # skip adding to rows
                    # detect progress change
                    prev_done = last_progress.get(filename, 0)
                    delta = done - prev_done
                    if delta > 0:
                        change_str = f"+{delta} 📈"
                    elif delta == 0:
                        change_str = "0 ➖"
                    else:
                        change_str = ""  # backward or reset
                    last_progress[filename] = done  # update snapshot
    
                    # check recent changes
                    mtime = datetime.fromtimestamp(os.path.getmtime(filepath))
                    recent = datetime.now() - timedelta(minutes=1.5)
                    is_recent = mtime >= recent
                    recent_flag = "✨" if is_recent else ""
    
                    if 'exc' in err.lower() or 'err' in err.lower():
                        message, emoji, color, group = err, f"❌{recent_flag}", RED, 2
                    elif done < total :
                        message, emoji, color, group = "Running", f"⏳{recent_flag}", YELLOW, 1
                    elif err == 'Done Already':
                        message, emoji, color, group = "Done Already", f"💤{recent_flag}", DARK_GREEN, 4
                    else:
                        message, emoji, color, group = "OK", f"✅{recent_flag}", GREEN, 3
    
                    rows.append((group, not is_recent, -percent, filename, message, progress_str, change_str, emoji, color))
    
            rows.sort()
    
            # print table
            running_count = sum(1 for r in rows if r[4] == "Running")
            pre_done_count = sum(1 for r in rows if r[4] == "Done Already")
            error_count   = sum(1 for r in rows if r[4] != "Running" and r[4] != "OK")  # catches ❌
            ok_count      = sum(1 for r in rows if r[4] == "OK")
    
            # print table header
            clear_output(wait=True)
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"\n📅 Snapshot Time: {now} \n\nSeasons : | ⏳: {running_count} | ❌: {error_count} | ✅: {ok_count} | 💤: {pre_done_count}\n ")
            print(f"{'S.No.':<6} {'Filename':<65} {'Message':<25} {'Progress':<20} {'+/-':<10} {'Status':<10}")
            print("-" * 138)
            
            for idx, (_, _, _, filename, message, progress_str, change_str, emoji, color) in enumerate(rows, start=1):
                row = f"{len(rows) - idx + 1:<4} {' : '.join(filename.split('_-_')[:2]).replace('_',' '):<65} {message:<25} {progress_str:<20} {change_str:<13} {emoji:<10}"
                print(f"{color}{row}{RESET}")
                
            time.sleep(interval)
    except KeyboardInterrupt:
        print("\nStopped.")

# if __name__ == "__main__":
watch_directory(r"D:\runtime_logs", interval=3)



📅 Snapshot Time: 2026-01-07 10:55:03 

Seasons : | ⏳: 21 | ❌: 27 | ✅: 23 | 💤: 26
 
S.No.  Filename                                                          Message                   Progress             +/-        Status    
------------------------------------------------------------------------------------------------------------------------------------------
71   Allsvenskan : 2026                                                Running                   0.00% (0/1)          0 ➖           ⏳         
70   Besta deild karla : 2026                                          Running                   0.00% (0/1)          0 ➖           ⏳         
69   Brasileiro Serie A : 2026                                         Running                   0.00% (0/1)          0 ➖           ⏳         
68   Brasileiro Serie B : 2026                                         Running                   0.00% (0/1)          0 ➖           ⏳         
67   CONCACAF Champions Cup : 2026                             

In [ ]:
MLS 24 should have more than 46 matches info

In [7]:
import pandas as pd

In [ ]:
retry = [{"comp" :x.split(",")[0].strip(" "),"season":x.split(",")[-1].strip(" ").replace(" ","/")} for x in """
71   Allsvenskan : 2026                                                Running                   0.00% (0/1)          0 ➖           ⏳         
70   Besta deild karla : 2026                                          Running                   0.00% (0/1)          0 ➖           ⏳         
69   Brasileiro Serie A : 2026                                         Running                   0.00% (0/1)          0 ➖           ⏳         
68   Brasileiro Serie B : 2026                                         Running                   0.00% (0/1)          0 ➖           ⏳         
67   CONCACAF Champions Cup : 2026                                     Running                   0.00% (0/1)          0 ➖           ⏳         
66   Canadian Premier League : 2026                                    Running                   0.00% (0/1)          0 ➖           ⏳         
65   Categoria Primera A : 2026                                        Running                   0.00% (0/1)          0 ➖           ⏳         
64   Chilean Primera Division : 2026                                   Running                   0.00% (0/1)          0 ➖           ⏳         
63   Copa Libertadores : 2026                                          Running                   0.00% (0/1)          0 ➖           ⏳         
62   Copa Sudamericana : 2026                                          Running                   0.00% (0/1)          0 ➖           ⏳         
61   Division Profesional : 2026                                       Running                   0.00% (0/1)          0 ➖           ⏳         
60   Eliteserien : 2026                                                Running                   0.00% (0/1)          0 ➖           ⏳         
59   Ireland Premier Division : 2026                                   Running                   0.00% (0/1)          0 ➖           ⏳         
58   J1 League : 2026 2027                                             Running                   0.00% (0/1)          0 ➖           ⏳         
57   Liga 1 : 2026                                                     Running                   0.00% (0/1)          0 ➖           ⏳         
56   Liga Pro : 2026                                                   Running                   0.00% (0/1)          0 ➖           ⏳         
55   Liga Profesional : 2026                                           Running                   0.00% (0/1)          0 ➖           ⏳         
54   Major League Soccer : 2026                                        Running                   0.00% (0/1)          0 ➖           ⏳         
53   Primera Division : 2026                                           Running                   0.00% (0/1)          0 ➖           ⏳         
52   Veikkausliiga : 2026                                              Running                   0.00% (0/1)          0 ➖           ⏳         
51   Virsliga : 2026 
""".split("\n") if x ]
retry

In [12]:
Romainian Liga 1 21/22


In [29]:
Season.objects.all()[105].competition.__dict__

{'_state': <django.db.models.base.ModelState at 0x213fb740590>,
 'id': 15,
 'confederation': 'CONMEBOL',
 'country': 'CONMEBOL',
 'competition_name': 'Copa Sudamericana',
 'name_scoresaway': 'CONMEBOL Sudamericana',
 'name_fotmob': 'Copa Sudamericana',
 'competition_format': 'H',
 'competition_type': 'I',
 'season_start': 3,
 'season_end': 12,
 'event_data_available': True,
 'event_data_url': 'https://www.scoresway.com/en_GB/soccer/conmebol-sudamericana-2025/bfz5665mze2yubs4s5t21f66s/fixtures',
 'shot_data_available': False,
 'shot_data_url': ''}

In [30]:
c = 0
retry_ones = []
for element in retry :
    try :
        tmp = {}
        obj = Season.objects.get(name=element.get('season'),competition__competition_name=element.get('comp')) 
        tmp['confederation'] = obj.competition.confederation
        tmp['region'] = obj.competition.country
        tmp['competition'] = obj.competition.competition_name
        tmp['season'] = obj.name
        retry_ones.append(tmp)
    except:
        c+=1
        print(c,"element Not found : ",element)

In [34]:
pd.DataFrame(retry_ones).to_excel("Last_Failures.xlsx",index=False)

In [ ]:
Season